In [1]:
from caffe2.python import workspace, model_helper
import numpy as np

x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.14190898 0.58269195]
  [0.14804004 0.19576225]
  [0.53221215 0.84280765]]

 [[0.74237037 0.56679609]
  [0.21686399 0.67209763]
  [0.15254745 0.5789201 ]]

 [[0.70896036 0.69264432]
  [0.09577659 0.73223972]
  [0.57646389 0.97076741]]

 [[0.17036442 0.36137439]
  [0.73471917 0.56436136]
  [0.7520975  0.92764818]]]
(4, 3, 2)
[[[0.14190898 0.58269195]
  [0.14804004 0.19576225]
  [0.53221215 0.84280765]]

 [[0.74237037 0.56679609]
  [0.21686399 0.67209763]
  [0.15254745 0.5789201 ]]

 [[0.70896036 0.69264432]
  [0.09577659 0.73223972]
  [0.57646389 0.97076741]]

 [[0.17036442 0.36137439]
  [0.73471917 0.56436136]
  [0.7520975  0.92764818]]]


In [2]:
data = np.random.rand(16,100).astype(np.float32)
label = (np.random.rand(16)*10).astype(np.int32)

workspace.FeedBlob("data",data)
workspace.FeedBlob("label",label)

True

In [3]:
model = model_helper.ModelHelper(name = "net")

In [4]:
weights = model.param_init_net.XavierFill([],"fc_w",shape=[10,100])
bias = model.param_init_net.ConstantFill([],"fc_b",shape=[10,])

In [5]:
fc1 = model.net.FC(["data","fc_w","fc_b"],"fc1")
pred = model.net.Sigmoid(fc1,"pred")
softmax,loss = model.net.SoftmaxWithLoss([pred,"label"],["softmax","loss"])
model.AddGradientOperators([loss])
print(model.net.Proto())

name: "net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



In [6]:
workspace.RunNetOnce(model.param_init_net)

True

In [7]:
workspace.CreateNet(model.net)

True

In [8]:
for _ in range(100):
    data = np.random.rand(16,100).astype(np.float32)
    label = (np.random.rand(16)*10).astype(np.int32)
    
    workspace.FeedBlob("data",data)
    workspace.FeedBlob("label",label)
    
    workspace.RunNet(model.name,10)


In [11]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.09665411 0.11637841 0.08637363 0.09426858 0.09424146 0.08368632
  0.10378861 0.09185072 0.13255095 0.10020716]
 [0.10222119 0.10569521 0.07872098 0.10924561 0.09921387 0.08424808
  0.09895405 0.09063099 0.12431713 0.10675295]
 [0.10619329 0.11566251 0.07528977 0.10254584 0.09884842 0.07508507
  0.09696864 0.08613555 0.13549322 0.10777763]
 [0.10433014 0.11142292 0.07783844 0.10616309 0.10503541 0.07584623
  0.10450024 0.08888117 0.1294777  0.09650476]
 [0.092739   0.11764689 0.08240782 0.0875389  0.09717013 0.08459236
  0.11166982 0.08977585 0.14096618 0.09549301]
 [0.11188678 0.11764981 0.0805137  0.10056552 0.09243735 0.08298058
  0.10554389 0.08386409 0.13213968 0.0924186 ]
 [0.10913911 0.11898073 0.08577812 0.09326787 0.08756094 0.08614404
  0.08795089 0.09862316 0.13471417 0.09784108]
 [0.11126507 0.12047957 0.07661424 0.10398193 0.09800398 0.07962672
  0.09421778 0.09432238 0.12741573 0.0940726 ]
 [0.09249195 0.11211536 0.08515187 0.09501904 0.11064223 0.0750877
  0.10363222 